In [5]:
import polars as pl
import os

In [6]:
in_file_path = os.path.join(".", "swimlane", "test_files", "schedule_swimlane_now.csv")
# schema_dict = {
#     "order_id": str,
#     "part_number": str,
#     "request_type": str,
#     "die_revision": str,
#     "configuration_id": str,
#     "platform": str,
#     "assigned_printer": str,
#     "printer_hood": str,
#     "printer_model": str,
#     "completion_date": str,
#     "job_number": str,
#     "print_number": str,
#     "qty_parts": str,
#     "plan_print_start_datetime": str,
#     "print_file": str,
#     "print_file_name_cfg": str,
#     "material_file_name_cfg": str,
#     "estimated_print_time": str,
#     "estimated_print_time_minutes": str,
#     "master_job_map_pk_id": str,
#     "estimated_plan_print_end_datetime": str,
#     "actual_print_start_datetime": str,
#     "actual_print_end_datetime": str,
#     "scrapped": str,
#     "duplicate_estimated_end_times": str,
# }
today_prints = pl.read_csv(in_file_path, separator=",")


In [7]:
today_prints_clean = (
    today_prints
    .filter(pl.col('scrapped').str.starts_with('No'))
    .select(
        'request_type',
        'part_number',
        'platform',
        'assigned_printer',
        'printer_hood',
        'printer_model',
        'job_number',
        'print_number',
        'qty_parts',
        'plan_print_start_datetime',
        'estimated_print_time_minutes',
        # 'scrapped'
    ).with_columns(
        pl.concat_str([
            pl.lit('"calc("'), 
            pl.col('estimated_print_time_minutes'), 
            pl.lit("% - 20px)")
        ], separator="").alias(
        "x_coord_style"
    ))
)
# today_prints_clean

In [15]:
prints_by_printer = [
    (
        str.strip(row.select(pl.first('assigned_printer')).to_dict(as_series=False)['assigned_printer'][0]), 
        str.strip(row.select(pl.first('printer_hood')).to_dict(as_series=False)['printer_hood'][0]), 
        str.strip(row.select(pl.first('printer_model')).to_dict(as_series=False)['printer_model'][0]), 
        # row.to_dicts()
        [
           print | {
               'prindex': prindex, 
               'x_coord_calc': "calc("+str(print["estimated_print_time_minutes"] / 17.5)+"% - 20px)"
            } for prindex, print in enumerate(row.to_dicts())
        ]
    )
    for row in today_prints_clean.sort(['plan_print_start_datetime', 'assigned_printer']).partition_by('assigned_printer')
]
prints_by_printer




[('P6-071',
  '5',
  'Core 550XT',
  [{'request_type': 'Production',
    'part_number': 9341028,
    'platform': 'Core 550X, Core 550XT, FORM 4, FORM 4L',
    'assigned_printer': 'P6-071    ',
    'printer_hood': '5',
    'printer_model': 'Core 550XT',
    'job_number': 6244708,
    'print_number': 50291,
    'qty_parts': 6,
    'plan_print_start_datetime': '2025-10-31 00:00:00',
    'estimated_print_time_minutes': 300,
    'x_coord_style': '"calc("300% - 20px)',
    'prindex': 0,
    'x_coord_calc': 'calc(17.142857142857142% - 20px)'},
   {'request_type': 'QC',
    'part_number': 9342004,
    'platform': 'Core 550XT',
    'assigned_printer': 'P6-071    ',
    'printer_hood': '5',
    'printer_model': 'Core 550XT',
    'job_number': 6247519,
    'print_number': 50579,
    'qty_parts': 8,
    'plan_print_start_datetime': '2025-10-31 05:45:00',
    'estimated_print_time_minutes': 360,
    'x_coord_style': '"calc("360% - 20px)',
    'prindex': 1,
    'x_coord_calc': 'calc(20.5714285714285

In [9]:
today_prints_clean.sort(['plan_print_start_datetime', 'assigned_printer']).partition_by('assigned_printer')

[shape: (2, 12)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ request_t ┆ part_numb ┆ platform  ┆ assigned_ ┆ … ┆ qty_parts ┆ plan_prin ┆ estimated ┆ x_coord_ │
 │ ype       ┆ er        ┆ ---       ┆ printer   ┆   ┆ ---       ┆ t_start_d ┆ _print_ti ┆ style    │
 │ ---       ┆ ---       ┆ str       ┆ ---       ┆   ┆ i64       ┆ atetime   ┆ me_minute ┆ ---      │
 │ str       ┆ i64       ┆           ┆ str       ┆   ┆           ┆ ---       ┆ s         ┆ str      │
 │           ┆           ┆           ┆           ┆   ┆           ┆ str       ┆ ---       ┆          │
 │           ┆           ┆           ┆           ┆   ┆           ┆           ┆ i64       ┆          │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ Productio ┆ 9341028   ┆ Core      ┆ P6-071    ┆ … ┆ 6         ┆ 2025-10-3 ┆ 300       ┆ "calc("3 │
 │ n         ┆           ┆ 550X,     ┆           ┆   ┆           ┆